<a href="https://colab.research.google.com/github/AlfonsoVasquezC/DataFiles/blob/main/SVD_Equipo52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Sinaí Avalos Rivera A01730466

*   Carlos Gustavo Lopez Lerma A01793468

*   Ricardo Morales Bustillos A01740032

*   Juan Alfonso Vásquez Chapoi A00820656

*   Ronald David Zúniga Sánchez A01686240
 


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [420]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

#**Ejercicio-1.** 

## Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega sus primeros 10 renglones. 

In [421]:
path_rating = 'https://raw.githubusercontent.com/AlfonsoVasquezC/DataFiles/main/rating_final.csv'

path_geoloc = 'https://raw.githubusercontent.com/AlfonsoVasquezC/DataFiles/main/geoplaces2.csv'

df_rating = pd.read_csv(path_rating, encoding = 'latin1')

df_geoloc = pd.read_csv(path_geoloc, encoding = 'latin1')

df_rating.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [422]:
df_geoloc.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [423]:
df_join = pd.merge(df_rating,df_geoloc[['placeID', 'name']], on='placeID')   # dataFrame

df_join.head(10)

,userID,placeID,rating,food_rating,service_rating,name
0,U1077,135085,2,2,2,Tortas Locas Hipocampo
1,U1108,135085,1,2,1,Tortas Locas Hipocampo
2,U1081,135085,1,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,2,Tortas Locas Hipocampo
4,U1134,135085,2,1,2,Tortas Locas Hipocampo
5,U1116,135085,2,2,2,Tortas Locas Hipocampo
6,U1132,135085,2,2,2,Tortas Locas Hipocampo
7,U1120,135085,0,0,0,Tortas Locas Hipocampo
8,U1036,135085,2,2,2,Tortas Locas Hipocampo
9,U1084,135085,2,2,2,Tortas Locas Hipocampo


In [424]:
UtilityMatrix = df_join.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

UtilityMatrix.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [425]:
UtilityMatrix.shape

(138, 129)

#**Ejercicio-2.**

## Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos sus valores singulares. En particular indica la cantidad de valores singulares obtenidos. 

### como queremos obtener la información latente en relación a los restaurantes, así que necesitamos que los renglones de la matriz de utilidad sean los restaurantes. Así, antes de aplicar la factorización SVD para reducción de la dimensionalidad debemos tomar la traspuesta de dicha matriz de utilidad, ya que por el momento lo que tenemos son usuarios en los renglones y restaurantes en los columnas.

In [426]:
UMT= UtilityMatrix.T
UMT.shape

(129, 138)

In [427]:
# El número de componentes debe ser menor que el número de características (features/usuarios), que son 138:
SVD = TruncatedSVD(n_components=137, random_state=1)  
SVD.fit(UMT)

num_sv = 10   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada/reducida con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada/reducida con los primeros 10 vectores singulares:
75.6%


In [428]:
num_sv = 10

SVD = TruncatedSVD(n_components=num_sv, random_state=1)

SVD_Matrix = SVD.fit_transform(UMT)
SVD_Matrix.shape

(129, 10)

#**Ejercicio-3.**

## Usando la métrica de correlación de Pearon, obtener las 10 mejores recomendaciones de este modelo no supervisado con base a la información de alguien que evaluó bien de manera general al “Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones. 

In [429]:
def restaurant_recom(UMT, SVD_Matrix, Restaurante, Topn):
  corrMx = np.corrcoef(SVD_Matrix)
  me_gusta = f"{Restaurante}"

  names = UtilityMatrix.columns
  names_list = list(names)
  id_megusta = names_list.index(me_gusta)

  corr_recomienda = corrMx[id_megusta]

  original_dic = {}

  for i in range(len(names)):
    original_dic[names[i]] = [corr_recomienda[i]]

  original_df = pd.DataFrame.from_dict(original_dic)
  original_df = original_df.T.sort_values(by=[0], ascending = False)
  filtered_df = original_df.iloc[1:,:]
  filtered_df = filtered_df.head(Topn)
  filtered_df.reset_index(inplace=True)
  filtered_df.rename(columns = {'index': 'Restaurante', 0: 'Correlación'}, inplace = True)
  
  return filtered_df

In [430]:
recomendaciones = restaurant_recom(UMT, SVD_Matrix, 'Subway', 10)
recomendaciones

,Restaurante,Correlación
0,Rincon del Bife,0.999991
1,Paniroles,0.999977
2,Restaurant Teely,0.999956
3,Restaurant and Bar and Clothesline Carlos N Ch...,0.999927
4,Mariscos Tia Licha,0.999906
5,Chilis Cuernavaca,0.999904
6,Mikasa,0.999879
7,Vips,0.999704
8,Log Yin,0.999397
9,Sanborns Casa Piedra,0.999228


#**Ejercicio-4.**

## Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares encontrado y la cantidad de información que contienen. 

### Redefinimos el modelo inicial tomando como base 10 componentes

In [431]:
# El número de componentes debe ser menor que el número de características (features/usuarios), que son 138:
SVD = TruncatedSVD(n_components=137, random_state=1)  
SVD.fit(UMT)

num_sv = 10   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada/reducida con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))


Cantidad de información simplificada/reducida con los primeros 10 vectores singulares:
75.6%


### Creamos un modelo nuevo reducido con 8 componentes

In [432]:
# El número de componentes debe ser menor que el número de características (features/usuarios), que son 138:
SVD_Reduced = TruncatedSVD(n_components=137, random_state=1)  
SVD_Reduced.fit(UMT)

num_sv_Reduced = 8   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada/reducida con los primeros %d vectores singulares:' % num_sv_Reduced)
print('%.1f%%' %  (100 * (1- (SVD_Reduced.singular_values_[0:num_sv_Reduced]).sum() / (SVD_Reduced.singular_values_).sum())))


Cantidad de información simplificada/reducida con los primeros 8 vectores singulares:
79.3%


In [433]:
SVD_Reduced = TruncatedSVD(n_components=num_sv_Reduced, random_state=1)

SVD_Matrix_Reduced = SVD_Reduced.fit_transform(UMT)
SVD_Matrix_Reduced.shape

(129, 8)

In [434]:
recomendaciones = restaurant_recom(UMT, SVD_Matrix_Reduced, 'Subway', 10)
recomendaciones

,Restaurante,Correlación
0,Rincon del Bife,0.999994
1,Log Yin,0.999994
2,Restaurant Teely,0.999992
3,Sanborns Casa Piedra,0.999988
4,Paniroles,0.999988
5,Vips,0.999984
6,Restaurant los Pinos,0.999984
7,Restaurant and Bar and Clothesline Carlos N Ch...,0.999981
8,Mariscos Tia Licha,0.999970
9,Mikasa,0.999958


In [435]:
comparación_df = restaurant_recom(UMT, SVD_Matrix, 'Subway', 10)


comparación_df['Nuevos Restaurantes'] = restaurant_recom(UMT, SVD_Matrix_Reduced, 'Subway', 10)['Restaurante']
comparación_df['Nuevos Correlaciones'] = restaurant_recom(UMT, SVD_Matrix_Reduced, 'Subway', 10)['Correlación']

comparación_df

,Restaurante,Correlación,Nuevos Restaurantes,Nuevos Correlaciones
0,Rincon del Bife,0.999991,Rincon del Bife,0.999994
1,Paniroles,0.999977,Log Yin,0.999994
2,Restaurant Teely,0.999956,Restaurant Teely,0.999992
3,Restaurant and Bar and Clothesline Carlos N Ch...,0.999927,Sanborns Casa Piedra,0.999988
4,Mariscos Tia Licha,0.999906,Paniroles,0.999988
5,Chilis Cuernavaca,0.999904,Vips,0.999984
6,Mikasa,0.999879,Restaurant los Pinos,0.999984
7,Vips,0.999704,Restaurant and Bar and Clothesline Carlos N Ch...,0.999981
8,Log Yin,0.999397,Mariscos Tia Licha,0.999970
9,Sanborns Casa Piedra,0.999228,Mikasa,0.999958


In [448]:
comparación_df = restaurant_recom(UMT, SVD_Matrix, 'Subway', 10)


comparación_df['Nuevos Restaurantes'] = restaurant_recom(UMT, SVD_Matrix_Reduced, 'Subway', 10)['Restaurante']
comparación_df['Nuevos Correlaciones'] = restaurant_recom(UMT, SVD_Matrix_Reduced, 'Subway', 10)['Correlación']

comparación_df[['Restaurante', 'Nuevos Restaurantes']]

rest = comparación_df['Restaurante']
rest_nuev = comparación_df['Nuevos Restaurantes']

contador = 0
estados = {}

for i in range(len(rest)):
  
  for j in range(len(rest_nuev)):
    
    if rest_nuev[j] == rest[i]:
      print(f"{rest[i]} fue encontrado en los nuevos restaurantes" )
      encontrado = True
      estados[f"{rest_nuev[i]}"] = True
      contador += 1
    else:
      continue

  try:
    estados[f"{rest_nuev[i]}"]
  except:
    print("\n")
    print(f"{rest[i]} NO fue encontrado en los nuevos restaurantes")
    print("\n")
    continue

print("\nEncontramos", contador, "Restaurantes\n")

if contador >= 9:
  print("Encontramos el minimo de 9 mismas Restaurantes originales")
else:
  ("No encontramos el minimo de las 9 mismas Restaurantes originales")


Rincon del Bife fue encontrado en los nuevos restaurantes
Paniroles fue encontrado en los nuevos restaurantes
Restaurant Teely fue encontrado en los nuevos restaurantes
Restaurant and Bar and Clothesline Carlos N Charlies fue encontrado en los nuevos restaurantes
Mariscos Tia Licha fue encontrado en los nuevos restaurantes


Chilis Cuernavaca NO fue encontrado en los nuevos restaurantes


Mikasa fue encontrado en los nuevos restaurantes
Vips fue encontrado en los nuevos restaurantes
Log Yin fue encontrado en los nuevos restaurantes
Sanborns Casa Piedra fue encontrado en los nuevos restaurantes

Encontramos 9 Restaurantes

Encontramos el minimo de 9 mismas Restaurantes originales


### Para este caso en particular pudimos trabajar con 8 componentes (reduciendo en un 79.3% la información y logramos encontrar los mismos 9 restaurantes que tomando 10 componentes (reduciendo 75.6% de la información).

### Es decir, con un 3.7% menos información logramos obtener los mismos 9 restaurantes.

#**Ejercicio-5.**

## Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones.

In [437]:
"tacos de barbacoa enfrente del Tec"

'tacos de barbacoa enfrente del Tec'

### Redefinimos el modelo inicial tomando como base 10 componentes

In [438]:
# El número de componentes debe ser menor que el número de características (features/usuarios), que son 138:
SVD_Tec = TruncatedSVD(n_components=137, random_state=1)  
SVD_Tec.fit(UMT)

num_sv_Tec = 10   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada/reducida con los primeros %d vectores singulares:' % num_sv_Tec)
print('%.1f%%' %  (100 * (1- (SVD_Tec.singular_values_[0:num_sv_Tec]).sum() / (SVD_Tec.singular_values_).sum())))

Cantidad de información simplificada/reducida con los primeros 10 vectores singulares:
75.6%


In [439]:
SVD_Tec = TruncatedSVD(n_components=num_sv_Tec, random_state=1)

SVD_Tec_Matrix = SVD_Tec.fit_transform(UMT)


### Creamos un modelo nuevo reducido con 4 componentes

In [449]:
# El número de componentes debe ser menor que el número de características (features/usuarios), que son 138:
SVD_Tec_Reduced = TruncatedSVD(n_components=137, random_state=1)  
SVD_Tec_Reduced.fit(UMT)

num_sv_Tec_Reduced = 4   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada/reducida con los primeros %d vectores singulares:' % num_sv_Tec_Reduced)
print('%.1f%%' %  (100 * (1- (SVD_Tec_Reduced.singular_values_[0:num_sv_Tec_Reduced]).sum() / (SVD_Tec_Reduced.singular_values_).sum())))

Cantidad de información simplificada/reducida con los primeros 4 vectores singulares:
87.7%


In [450]:
SVD_Tec_Reduced = TruncatedSVD(n_components=num_sv_Tec_Reduced, random_state=1)

SVD_Tec_Reduced_Matrix = SVD_Tec_Reduced.fit_transform(UMT)

In [451]:
recomendaciones = restaurant_recom(UMT, SVD_Tec_Matrix, "tacos de barbacoa enfrente del Tec", 10)
recomendaciones

,Restaurante,Correlación
0,tacos abi,0.999956
1,Taqueria EL amigo,0.999901
2,vips,0.999781
3,little pizza Emilio Portes Gil,0.999704
4,Pollo_Frito_Buenos_Aires,0.999520
5,puesto de gorditas,0.999434
6,carnitas_mata,0.999413
7,palomo tec,0.999407
8,Hamburguesas La perica,0.998270
9,Little Cesarz,0.998220


In [452]:
comparación_df = restaurant_recom(UMT, SVD_Matrix, 'tacos de barbacoa enfrente del Tec', 10)


comparación_df['Nuevos Restaurantes'] = restaurant_recom(UMT, SVD_Tec_Reduced_Matrix, 'tacos de barbacoa enfrente del Tec', 10)['Restaurante']
comparación_df['Nuevos Correlaciones'] = restaurant_recom(UMT, SVD_Tec_Reduced_Matrix, 'tacos de barbacoa enfrente del Tec', 10)['Correlación']

comparación_df

,Restaurante,Correlación,Nuevos Restaurantes,Nuevos Correlaciones
0,tacos abi,0.999956,little pizza Emilio Portes Gil,0.999999
1,Taqueria EL amigo,0.999901,vips,0.999999
2,vips,0.999781,tacos abi,0.999998
3,little pizza Emilio Portes Gil,0.999704,puesto de gorditas,0.999996
4,Pollo_Frito_Buenos_Aires,0.999520,Pollo_Frito_Buenos_Aires,0.999996
5,puesto de gorditas,0.999434,Taqueria EL amigo,0.999995
6,carnitas_mata,0.999413,carnitas_mata,0.999994
7,palomo tec,0.999407,Gorditas Dona Tota,0.999994
8,Hamburguesas La perica,0.998270,palomo tec,0.999987
9,Little Cesarz,0.998220,Little Cesarz,0.999984


In [453]:
rest = comparación_df['Restaurante']
rest_nuev = comparación_df['Nuevos Restaurantes']

contador = 0
estados = {}

for i in range(len(rest)):
  
  for j in range(len(rest_nuev)):
    
    if rest_nuev[j] == rest[i]:
      print(f"{rest[i]} fue encontrado en los nuevos restaurantes" )
      encontrado = True
      estados[f"{rest_nuev[i]}"] = True
      contador += 1
    else:
      continue

  try:
    estados[f"{rest_nuev[i]}"]
  except:
    print("\n")
    print(f"{rest[i]} NO fue encontrado en los nuevos restaurantes")
    print("\n")
    continue

print("\nEncontramos", contador, "Restaurantes\n")

if contador >= 9:
  print("Encontramos el minimo de 9 mismas Restaurantes originales")
else:
  ("No encontramos el minimo de las 9 mismas Restaurantes originales")

tacos abi fue encontrado en los nuevos restaurantes
Taqueria EL amigo  fue encontrado en los nuevos restaurantes
vips fue encontrado en los nuevos restaurantes
little pizza Emilio Portes Gil fue encontrado en los nuevos restaurantes
Pollo_Frito_Buenos_Aires fue encontrado en los nuevos restaurantes
puesto de gorditas fue encontrado en los nuevos restaurantes
carnitas_mata fue encontrado en los nuevos restaurantes
palomo tec fue encontrado en los nuevos restaurantes


Hamburguesas La perica NO fue encontrado en los nuevos restaurantes


Little Cesarz fue encontrado en los nuevos restaurantes

Encontramos 9 Restaurantes

Encontramos el minimo de 9 mismas Restaurantes originales


### Para este caso en particular pudimos trabajar con 4 componentes (reduciendo en un 87.7% la información y logramos encontrar los mismos 9 restaurantes que tomando 10 componentes (reduciendo 75.6% de la información).

### Es decir, con un 9% menos información logramos obtener 9 de los 10 restaurantes obtenidos con 10 componentes; solamente que en el caso de Subway no fue así. Esto se debe a que existe mayor relación entre estos restaurantes con los tacos enfrente a la biblioteca del tec que con los otros 10 y un Subway.

#**Ejercicio-6.**

## Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada a la comida (food_rating). Incluye tus conclusiones. 


In [445]:
None

#**Ejercicio-7.**

## Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada al servicio (service_rating). Incluye tus conclusiones. 

In [446]:
None

#**Ejercicio-8.**

##  Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos. 

In [447]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**